In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, MultiHeadAttention,Input
import pandas as pd
import numpy as np
import json
from transformers import BertModel,AutoTokenizer
from sklearn.preprocessing import LabelEncoder
import random

2025-02-16 21:35:20.397929: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 21:35:20.409886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-16 21:35:20.421123: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-16 21:35:20.423800: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 21:35:20.434641: I tensorflow/core/platform/cpu_feature_guar

In [2]:
with open('chat-bot-data.json') as intents:
    data=json.load(intents)

In [3]:
data['intents'][10]

{'tag': 'sharjah_currency',
 'patterns': ['Which currency is used in Sharjah?',
  'What is the official currency of Sharjah?',
  'What money is used in Sharjah?',
  'What type of currency does Sharjah use?',
  'What is the legal tender in Sharjah?',
  'Which monetary unit is used in Sharjah?',
  'What is the name of the currency in Sharjah?',
  'What currency do people use in Sharjah?',
  'What is the standard currency in Sharjah?',
  'What is the primary currency used in Sharjah?',
  'What kind of money is accepted in Sharjah?',
  'What is the national currency of Sharjah?',
  'Which banknotes and coins are used in Sharjah?',
  'What is the official form of money in Sharjah?',
  'What currency should I bring to Sharjah?',
  'In what currency are prices listed in Sharjah?',
  'What is the exchange currency in Sharjah?',
  'What is the main form of payment in Sharjah?',
  'What currency do businesses in Sharjah accept?',
  'What currency should tourists use in Sharjah?'],
 'responses': 

In [4]:
tags=[]
inputs=[]
responses={}
for intent in data['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        inputs.append(lines)
        tags.append(intent['tag'])

In [5]:
df=pd.DataFrame({'Inputs':inputs,'tags':tags})

In [6]:
df

,Inputs,tags
0,What are some fun activities to do in Sharjah?,sharjah_activities
1,What can I do for fun in Sharjah?,sharjah_activities
2,Any recommendations for activities in Sharjah?,sharjah_activities
3,What are the best things to do in Sharjah?,sharjah_activities
4,What activities are popular in Sharjah?,sharjah_activities
...,...,...
275,What are the best CBSE schools in Sharjah?,sharjah_education
276,Which schools in Sharjah offer high-quality ed...,sharjah_education
277,What are the best schools in Sharjah for prima...,sharjah_education
278,Which schools in Sharjah are best for secondar...,sharjah_education


In [7]:
max_length=df['Inputs'].str.len().max()

In [8]:
max_length

70

In [9]:
df['tags'].value_counts()

tags
sharjah_activities           20
sharjah_historical_places    20
sharjah_family_activities    20
sharjah_shopping             20
sharjah_food                 20
sharjah_nature               20
sharjah_events               20
sharjah_accommodation        20
sharjah_weather              20
sharjah_language             20
sharjah_currency             20
sharjah_safety               20
sharjah_health               20
sharjah_education            20
Name: count, dtype: int64

In [10]:
def tokenize_data(tokenizer, texts):
  tokenized_texts = []
  for text in texts:
    tokens = tokenizer(text, padding='max_length', truncation=True, max_length=max_length)['input_ids']
    tokenized_texts.append(tokens)

  return np.array(tokenized_texts)

In [11]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
X =tokenize_data(tokenizer, df['Inputs'])
X.shape

(280, 70)

In [12]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
embedding_layer = Embedding(
    input_dim=bert_model.config.vocab_size,
    output_dim=bert_model.config.hidden_size,
    weights=[bert_model.embeddings.word_embeddings.weight.detach()],
    trainable=False)

I0000 00:00:1739730927.494169    2522 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739730927.495075    2522 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739730927.495107    2522 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739730927.498506    2522 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1739730927.498539    2522 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [13]:
le=LabelEncoder()
y=le.fit_transform(df['tags'])

In [14]:
outputlen=le.classes_.shape[0]
print(outputlen)

14


In [28]:
num_heads = 8
attention_dim = 128
input_data = tf.keras.Input(shape=(X.shape[1],))
emb = embedding_layer(input_data)
attention = MultiHeadAttention(num_heads=num_heads, key_dim=attention_dim, value_dim=attention_dim)
attention_output = attention(query=emb , key=emb, value=emb)
lstm_output = LSTM(128)(attention_output)
output = Dense(outputlen, activation='softmax')(lstm_output)
model = tf.keras.Model(inputs=input_data, outputs=output)

In [29]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 70)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 70, 768)   │ 23,440,896 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 70, 768)   │  3,149,568 │ embedding[3][0],  │
│ (MultiHeadAttentio… │                   │            │ embedding[3][0],  │
│                     │                   │            │ embedding[3][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 128)       │    459,264 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 14)        │      1,806 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 27,051,534 (103.19 MB)

 Trainable params: 3,610,638 (13.77 MB)

 Non-trainable params: 23,440,896 (89.42 MB)

In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [31]:
model.fit(X, y, epochs=20,batch_size=16,)


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.0897 - loss: 2.7211
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0693 - loss: 2.7294
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0550 - loss: 2.6754
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0630 - loss: 2.6344
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1021 - loss: 2.5766
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2480 - loss: 2.2227
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3085 - loss: 1.9761
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2804 - loss: 1.8973
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.3683 - loss: 1.6785
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4375 - loss: 1.5694
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4502 - loss: 1.5056
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy:

In [32]:
model.fit(X, y, epochs=10,batch_size=4,)


Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7923 - loss: 0.5265
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6827 - loss: 1.1642
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7487 - loss: 0.7345
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8448 - loss: 0.4934
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6152 - loss: 1.0681
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8513 - loss: 0.5855
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8994 - loss: 0.4472
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9072 - loss: 0.3545
Epoch 9/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9422 - loss: 0.2747
Epoch 10/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9434 - loss: 0.2411


In [33]:
model.fit(X, y, epochs=10,batch_size=8,)


Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9602 - loss: 0.2066
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9772 - loss: 0.1511
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9620 - loss: 0.1316
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9391 - loss: 0.2060
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9688 - loss: 0.0942
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9544 - loss: 0.1378
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9918 - loss: 0.0977
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9857 - loss: 0.0609
Epoch 9/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9994 - loss: 0.0556
Epoch 10/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9945 - loss: 0.0485


In [34]:
model.save('intents-chatbot.keras')

In [35]:
def chat():
    while True:
        Input = (input('You: '))
        Input = tokenize_data(tokenizer, np.array([Input]))
        prediction = model.predict(Input)
        output = prediction.argmax()
        response_tag = le.inverse_transform([output])[0]
        print(f"Predicted tag: {response_tag}") 
        responses = next(intent['responses'] for intent in data["intents"] if intent['tag'] == response_tag)
        print('Chatbot:', random.choice(responses))

In [36]:
chat()

You:  What activities are popular in Sharjah?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted tag: sharjah_activities
Chatbot: Visit the Sharjah Classic Cars Museum for a trip down memory lane.


You:  What activities is popular in Sharjah?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted tag: sharjah_activities
Chatbot: Visit the Al Majaz Waterfront for live events and entertainment.


You:  Which hotels in Sharjah are the best?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted tag: sharjah_accommodation
Chatbot: For a romantic stay, try the Al Bait Sharjah or the Chedi Khorfakkan Resort & Spa.


You:  Which time of year are best for a holiday in Sharjah?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted tag: sharjah_weather
Chatbot: The best time to visit Sharjah for outdoor activities is from November to March when the weather is cooler.


KeyboardInterrupt: Interrupted by user